### Kaggle Knowledge Competition: Titanic - Machine Learning from Disaster

My solution attempt at predicting which passengers survived the Titanic shipwreck.

Data and additional information is available at [Kaggle](https://www.kaggle.com/c/titanic)

In [1]:
# matplotlib inline plotting
%matplotlib inline
# make inline plotting higher resolution
%config InlineBackend.figure_format = 'svg'

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('dark_background')